# Low-level Python

# PyCUDA (and PyOpenCL): arbitrary GPU programming

The lowest of low-level programming— programming and running accelerator devices— can be done from the comfort of a notebook.

In [42]:
import pycuda
import pycuda.autoinit
import pycuda.driver
import pycuda.compiler

compiled_cuda = pycuda.compiler.SourceModule("""
__global__ void runs_on_gpu(float* p, float* px, float* py, float* pz) {
    const int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < 1000000) {
        p[i] = sqrt(px[i]*px[i] + py[i]*py[i] + pz[i]*pz[i]);
    }
}""")
runs_on_gpu = compiled_cuda.get_function("runs_on_gpu")

In [43]:
import numpy

px = pycuda.driver.In(numpy.random.normal(0, 10, 1000000).astype(numpy.float32))
py = pycuda.driver.In(numpy.random.normal(0, 10, 1000000).astype(dtype=numpy.float32))
pz = pycuda.driver.In(numpy.random.normal(0, 10, 1000000).astype(dtype=numpy.float32))
p = numpy.zeros(1000000, dtype=numpy.float32)

runs_on_gpu(pycuda.driver.Out(p), px, py, pz, block=(1024, 1, 1), grid=(1000000 // 1024 + 1, 1))

In [44]:
p

array([28.385084,  8.115287, 13.030323, ..., 16.229408, 11.36001 ,
       19.637793], dtype=float32)